# Métodos

In [1]:
import numpy as np

In [2]:
def solve_system_ts(A,b):
  n = len(b)
  b_cp = b.copy()
  x = np.zeros((n,1))
  for i in reversed(range(n)):
    for j in range(i+1, n):
      b_cp[i] -= x[j]*A[i,j]
    x[i] = b_cp[i]/A[i,i]

  return x

In [3]:
def solve_system_ti(A,b):
  n = len(b)
  b_cp = b.copy()
  x = np.zeros((n,1))
  for i in range(n):
    for j in range(0, i):
      b_cp[i] -= x[j]*A[i,j]
    x[i] = b_cp[i]/A[i,i]

  return x

In [4]:
def gauss_elimination(A, B):
  n = len(A)
  A_cp = A.copy()
  B_cp = B.copy()

  for i in range(n-1):
    pivot = A_cp[i,i]

    for j in range(i+1, n):
      m = (-1)*A_cp[j,i]/A_cp[i,i]
      A_cp[j,:] += A_cp[i,:]*m
      B_cp[j] += B_cp[i]*m

  return A_cp, B_cp

In [5]:
def gauss_jordan(A, B):
  n = len(A)
  A_cp = A.copy()
  B_cp = B.copy()

  for i in range(n):
    pivot = A_cp[i,i]
    A_cp[i,:] /= pivot
    B_cp[i] /= pivot

    for j in range(n):
      if j != i:
        m = (-1)*A_cp[j,i]/A_cp[i,i]
        A_cp[j,:] += A_cp[i,:]*m
        B_cp[j] += B_cp[i]*m

  return A_cp, B_cp

In [6]:
def lu_decomp(A):
  n = A.shape[0]
  U = A.copy()
  L = np.eye(n)

  for i in range(n):
    for j in range(i+1,n):
      coef = -U[j,i]/U[i,i]
      L[j,i] = -coef
      U[j] += U[i]*coef

  return L,U

In [7]:
def solve_lu(l, u, b):
  y = solve_system_ti(l,b)
  x = solve_system_ts(u,y)
  return x

In [8]:
def lines_criteria(matrix):
    diagonal = np.diag(matrix)
    row_sums = np.sum(matrix, axis=1) - diagonal
    return diagonal > row_sums

In [9]:
def sassenfeld(M):
  n = len(M)
  B = np.ones(n)
  for j in range(n):
    B[j] = sum(np.abs(M[j,i])*B[i] for i in range(n) if i != j)  / np.abs(M[j,j])

  return max(B)

In [10]:
def gauss_seidel(A,B, maxiter=100, tol=1e-8):
  C,g = A.copy(), B.copy()
  n = len(A)
  X = [np.zeros((n,1))]

  def dr(x_new,x):
    return np.max(np.abs(x_new - x)) / np.max(np.abs(x_new))

  for i in range(n):
    C[i] = -A[i]/A[i,i]
    C[i,i] = 0
    g[i] = B[i]/A[i,i]

  for _ in range(maxiter):
    x = X[-1].copy()
    for i in range(n):
      x[i] = C[i] @ x + g[i]
    X.append(x)

    if dr(X[-1], X[-2]) < tol:
      break

  return X

# (TEÓRICA) Questão 1

Resolva o seguinte sistema pelo método da decomposição LU (2,0 pontos).

\begin{cases}
\begin{aligned}
5x_1 + x_2 + x_3 &= 50 \\
-x_1 + 3x_2 - x_3 &= 10 \\
x_1 + 2x_2 + 10x_3 &= -30
\end{aligned}
\end{cases}

In [ ]:
A1 = np.array([
     [5,1,1],
     [-1,3,-1],
     [1,2,10]
], dtype='float')

B1 = np.array([50,10,-30],dtype='float').reshape(3,1)

In [ ]:
l, u = lu_decomp(A1)
x = solve_lu(l,u,B1)
x

array([[10.],
       [ 5.],
       [-5.]])

# (TEÓRICA) Questão 2

Dos seguintes três conjuntos de equações lineares, identifique qual(is)
conjunto(s) não pode(m) ser resolvido(s) usando um método iterativo como o de Gauss-Seidel. Mostre, utilizando qualquer número de iterações, que necessariamente sua solução não converge (3,0 pontos).

$$
\begin{array}{|c|c|c|}
\hline
\textbf{Conjunto Um} & \textbf{Conjunto Dois} & \textbf{Conjunto Três} \\
\hline
9x + 3y + z = 13 & x + y + 6z = 8 & -3x + 4y + 5z = 6 \\
-6x + 8z = 2 & x + 5y - z = 5 & -2x + 2y -3z = -3\\
2x + 5y - z = 6 & 4x + 2y -2z = 4 & 2y - z = 1\\
\hline
\end{array}
$$

## Conjunto Um

In [ ]:
set1 = np.array([
    [9,3,1],
    [-6,0,8],
    [2,5,-1]
], dtype='float')

set1_b = np.array([13,2,6],dtype='float').reshape(3,1)
set1[[1,2]] = set1[[2,1]]
set1_b[[1,2]] = set1_b[[2,1]]
print(lines_criteria(set1))
print(sassenfeld(set1)) # 0.44 -> Converge

[ True  True  True]
0.4444444444444444


In [ ]:
sol_set1 = gauss_seidel(set1,set1_b)
sol_set1[-1]

array([[1.],
       [1.],
       [1.]])

## Conjunto Dois

In [ ]:
set2 = np.array([
    [1,1,6],
    [1,5,-1],
    [4,2,-2]
], dtype='float')

set2_b = np.array([8,5,4],dtype='float').reshape(3,1)
set2[[0,2]] = set2[[2,0]]
set2_b[[0,2]] = set2_b[[2,0]]
print(lines_criteria(set2)) # Satisfaz critério das linhas -> converge
print(sassenfeld(set2))

[ True  True  True]
1.0


In [ ]:
sol_set2 = gauss_seidel(set2,set2_b)
sol_set2[-1]

array([[1.],
       [1.],
       [1.]])

## Conjunto Três

In [13]:
set3 = np.array([
    [-3,4,5],
    [-2,2,-3],
    [0,2,-1],
], dtype='float')

set3_b = np.array([6,-3,1], dtype='float').reshape(3,1)
print(lines_criteria(set3))
print(sassenfeld(set3)) # Nao converge (falha nos dois criterios)

[False  True False]
9.0


In [14]:
sol_set3 = gauss_seidel(set3,set3_b)
sol_set3[-1]

array([[-1.03196332e+88],
       [-1.69536831e+88],
       [-3.39073663e+88]])

# (PRÁTICA) Questão 3

Aplique o método da eliminação de Gauss ou a decomposição LU (à sua
escolha) para obter a inversa da matriz (3,0 pontos):

$$
A =
\begin{bmatrix}
4 & -1 & 0 & -1 & 0 & 0 \\
-1 & 4 & -1 & 0 & -1 & 0 \\
0 & -1 & 4 & 0 & 0 & -1 \\
-1 & 0 & 0 & 4 & -1 & 0 \\
0 & -1 & 0 & -1 & 4 & -1 \\
0 & 0 & -1 & 0 & -1 & 4 \\
\end{bmatrix}
$$

In [ ]:
A3 = np.array([
    [4,-1,0,-1,0,0],
    [-1,4,-1,0,-1,0],
    [0,-1,4,0,0,-1],
    [-1,0,0,4,-1,0],
    [0,-1,0,-1,4,-1],
    [0,0,-1,0,-1,4]
], dtype='float')

I = np.eye(6)
I1 = I[:,0].reshape(6,1)
I2 = I[:,1].reshape(6,1)
I3 = I[:,2].reshape(6,1)
I4 = I[:,3].reshape(6,1)
I5 = I[:,4].reshape(6,1)
I6 = I[:,5].reshape(6,1)

In [ ]:
a1,b1 = gauss_elimination(A3,I1)
x1 = solve_system_ts(a1,b1)

a2,b2 = gauss_elimination(A3,I2)
x2 = solve_system_ts(a2,b2)

a3,b3 = gauss_elimination(A3,I3)
x3 = solve_system_ts(a3,b3)

a4,b4 = gauss_elimination(A3,I4)
x4 = solve_system_ts(a4,b4)

a5,b5 = gauss_elimination(A3,I5)
x5 = solve_system_ts(a5,b5)

a6,b6 = gauss_elimination(A3,I6)
x6 = solve_system_ts(a6,b6)

In [ ]:
X3 = np.column_stack((x1,x2,x3,x4,x5,x6))
X3

array([[0.29482402, 0.0931677 , 0.02815735, 0.08612836, 0.04968944,
        0.0194617 ],
       [0.0931677 , 0.32298137, 0.0931677 , 0.04968944, 0.10559006,
        0.04968944],
       [0.02815735, 0.0931677 , 0.29482402, 0.0194617 , 0.04968944,
        0.08612836],
       [0.08612836, 0.04968944, 0.0194617 , 0.29482402, 0.0931677 ,
        0.02815735],
       [0.04968944, 0.10559006, 0.04968944, 0.0931677 , 0.32298137,
        0.0931677 ],
       [0.0194617 , 0.04968944, 0.08612836, 0.02815735, 0.0931677 ,
        0.29482402]])

# (PRÁTICA) QUESTÃO 4

Uma companhia de eletrônica produz transistores, resistores e chips de computador. Cada transistor usa quatro unidades de cobre, uma unidade de zinco e duas unidades de vidro. Cada resistor usa três, três e uma unidade de cada material, respectivamente, e cada chip de computador usa duas, uma, e três unidades desses materiais, respectivamente. Colocando essa informação em uma tabela, tem-se:

$$
\begin{array}{|c|c|c|c|}
\hline
\text{Componente} & \text{Cobre} & \text{Zinco} & \text{Vidro} \\
\hline
\text{Transistor} & 4 & 1 & 2 \\
\hline
\text{Resistor} & 3 & 3 & 1 \\
\hline
\text{Chip} & 2 & 1 & 3\\
\hline
\end{array}
$$

O fornecimento desses materiais varia de semana para semana. Assim, a companhia precisa
determinar uma meta de produção diferente para cada semana. Por exemplo, em uma
semana, a quantidade total de materiais disponíveis era 960 unidades de cobre, 510 unidades de zinco e 610 unidades de vidro. Determine o sistema de equações que modela essa meta de produção e use o algoritmo de Gauss-Jordan para determinar a solução (2,0 pontos)

$$4T + 3R + 2C = 960$$
$$1T + 3R + 1C = 510$$
$$2T + 1R + 3C = 610$$

In [ ]:
A4 = np.array([
    [4,3,2],
    [1,3,1],
    [2,1,3]
], dtype='float')

B4 = np.array([960,510,610], dtype='float').reshape(3,1)

In [ ]:
a4, b4 = gauss_jordan(A4,B4)

In [ ]:
b4

array([[120.],
       [100.],
       [ 90.]])

In [ ]:
a4

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])